# Load dim_shop từ Silver lên Gold

## Mục tiêu
- Load dữ liệu từ Silver.dim_shop → Gold.dim_shop
- Không cần cleaning, chỉ load trực tiếp
- Đảm bảo data quality cho Gold layer

## Quy trình 3 bước
1. **Load from Silver** - Đọc dữ liệu từ Silver database
2. **Load to Gold** - Load vào Gold database
3. **Create Dictionary** - Tạo data dictionary


## Bước 1: Import và Setup


In [4]:
# Import các thư viện cần thiết
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_SILVER = os.getenv("DB_SILVER")
DB_GOLD = os.getenv("DB_GOLD")

# Tạo kết nối tới Silver và Gold database
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")
gold_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_GOLD}")

print("Đã kết nối Silver và Gold database thành công")
print(f"Silver DB: {DB_SILVER}")
print(f"Gold DB: {DB_GOLD}")
print(f"Thời gian bắt đầu: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Đã kết nối Silver và Gold database thành công
Silver DB: winner_silver
Gold DB: winner_gold
Thời gian bắt đầu: 2025-10-18 00:16:42


## Bước 2: Load dữ liệu từ Silver


In [5]:
# Load dữ liệu dim_shop từ Silver database
print("=== LOADING DIM_SHOP FROM SILVER ===")

# Load toàn bộ bảng dim_shop
shop_df = pd.read_sql("SELECT * FROM dim_shops", silver_engine)

# Hiển thị thông tin cơ bản về dataset
print(f"Shape: {shop_df.shape}")
print(f"Columns: {list(shop_df.columns)}")
print(f"Memory usage: {shop_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Hiển thị sample data
print("\n=== SAMPLE DATA ===")
print(shop_df.head(3))

# Hiển thị data types
print("\n=== DATA TYPES ===")
print(shop_df.dtypes)

# Kiểm tra null values
print("\n=== NULL VALUES CHECK ===")
null_summary = shop_df.isnull().sum()
print("Null values per column:")
for col, null_count in null_summary.items():
    if null_count > 0:
        print(f"  {col}: {null_count} ({null_count/len(shop_df)*100:.1f}%)")
    else:
        print(f"  {col}: {null_count} (0.0%)")


=== LOADING DIM_SHOP FROM SILVER ===
Shape: (1, 4)
Columns: ['shop_id', 'shop_name', 'currency', 'avatar_url']
Memory usage: 0.00 MB

=== SAMPLE DATA ===
     shop_id     shop_name currency  \
0  230361475  Winner Group      VND   

                                          avatar_url  
0  https://content.pancake.vn/2-24/2024/2/4/e0b42...  

=== DATA TYPES ===
shop_id        int64
shop_name     object
currency      object
avatar_url    object
dtype: object

=== NULL VALUES CHECK ===
Null values per column:
  shop_id: 0 (0.0%)
  shop_name: 0 (0.0%)
  currency: 0 (0.0%)
  avatar_url: 0 (0.0%)


## Bước 3: Load vào Gold Database


In [6]:
# Load vào Gold Database
print("=== LOADING TO GOLD DATABASE ===")

from sqlalchemy.types import BigInteger, Integer, String, Text, DateTime, Float

# Định nghĩa schema mapping cho MySQL
dtype_mapping = {}

# ID fields - String (để xử lý mixed data types)
id_fields = ["shop_id", "owner_id"]
for field in id_fields:
    if field in shop_df.columns:
        dtype_mapping[field] = String(100)

# String fields - VARCHAR(255)
string_fields = ["shop_name", "shop_domain", "shop_status", "shop_type", "shop_category", "shop_description", "shop_address", "shop_city", "shop_country", "shop_phone", "shop_email"]
for field in string_fields:
    if field in shop_df.columns:
        dtype_mapping[field] = String(255)

# Text fields - Text()
text_fields = ["shop_description_long"]
for field in text_fields:
    if field in shop_df.columns:
        dtype_mapping[field] = Text()

# Numeric fields
float_fields = ["shop_rating", "shop_revenue"]
for field in float_fields:
    if field in shop_df.columns:
        dtype_mapping[field] = Float()

# Integer fields
int_fields = ["shop_followers", "shop_products_count", "shop_orders_count"]
for field in int_fields:
    if field in shop_df.columns:
        dtype_mapping[field] = Integer()

# DateTime fields
datetime_fields = ["shop_created_at", "shop_updated_at", "last_active_at"]
for field in datetime_fields:
    if field in shop_df.columns:
        dtype_mapping[field] = DateTime()

print(f"Schema mapping defined for {len(dtype_mapping)} columns")

# Load vào Gold database
table_name = "dim_shop"

try:
    shop_df.to_sql(
        table_name,
        con=gold_engine,
        if_exists="replace",  # Ghi đè dữ liệu cũ
        index=False,
        dtype=dtype_mapping
    )
    
    print(f"Đã load {shop_df.shape[0]} records vào Gold: {table_name}")
    
    # Verify load
    verification_query = f"SELECT COUNT(*) as count FROM {table_name}"
    verification_result = pd.read_sql(verification_query, gold_engine)
    loaded_count = verification_result['count'].iloc[0]
    
    print(f"Verification: {loaded_count} records trong database")
    print(f"Schema: {len(dtype_mapping)} columns với explicit typing")
    
    # Sample data từ database
    sample_query = f"SELECT * FROM {table_name} LIMIT 5"
    sample_db = pd.read_sql(sample_query, gold_engine)
    print(f"\nSample data từ Gold database:\n{sample_db}")
    
except Exception as e:
    print(f"Error loading to Gold: {str(e)}")
    raise

print(f"\nLoad dim_shop completed successfully!")
print(f"Ready for EDA and analytics")


=== LOADING TO GOLD DATABASE ===
Schema mapping defined for 2 columns
Đã load 1 records vào Gold: dim_shop
Verification: 1 records trong database
Schema: 2 columns với explicit typing

Sample data từ Gold database:
     shop_id     shop_name currency  \
0  230361475  Winner Group      VND   

                                          avatar_url  
0  https://content.pancake.vn/2-24/2024/2/4/e0b42...  

Load dim_shop completed successfully!
Ready for EDA and analytics


## Bước 4: Tạo Data Dictionary cho Gold


In [7]:
# Tạo Data Dictionary cho Gold
print("=== CREATING GOLD DATA DICTIONARY ===")

def get_business_meaning_vietnamese(column_name):
    """Get business meaning for each column in Vietnamese"""
    business_meanings = {
        # Định danh & Khóa
        "shop_id": "Mã định danh cửa hàng duy nhất (khóa chính)",
        "owner_id": "Mã chủ sở hữu cửa hàng",
        
        # Thông tin cơ bản
        "shop_name": "Tên cửa hàng",
        "shop_domain": "Tên miền cửa hàng",
        "shop_status": "Trạng thái cửa hàng (active, inactive, suspended)",
        "shop_type": "Loại cửa hàng (individual, business, enterprise)",
        "shop_category": "Danh mục cửa hàng",
        "shop_description": "Mô tả ngắn về cửa hàng",
        "shop_description_long": "Mô tả chi tiết về cửa hàng",
        
        # Địa chỉ và liên hệ
        "shop_address": "Địa chỉ cửa hàng",
        "shop_city": "Thành phố",
        "shop_country": "Quốc gia",
        "shop_phone": "Số điện thoại cửa hàng",
        "shop_email": "Email cửa hàng",
        
        # Thống kê
        "shop_rating": "Đánh giá trung bình của cửa hàng (1-5 sao)",
        "shop_followers": "Số lượng người theo dõi",
        "shop_products_count": "Số lượng sản phẩm trong cửa hàng",
        "shop_orders_count": "Số lượng đơn hàng đã xử lý",
        "shop_revenue": "Doanh thu cửa hàng (VND)",
        
        # Thời gian
        "shop_created_at": "Thời gian tạo cửa hàng",
        "shop_updated_at": "Thời gian cập nhật thông tin cuối cùng",
        "last_active_at": "Thời gian hoạt động cuối cùng"
    }
    return business_meanings.get(column_name, "Chưa định nghĩa ý nghĩa business")

# Tạo Data Dictionary
dict_data = []
for col in shop_df.columns:
    col_info = {
        "table_name": "dim_shop",
        "column_name": col,
        "dtype": str(shop_df[col].dtype),
        "sql_type": str(dtype_mapping.get(col, "Not defined")),
        "null_count": shop_df[col].isnull().sum(),
        "null_pct": round(shop_df[col].isnull().mean() * 100, 2),
        "unique_count": shop_df[col].nunique(),
        "sample_values": str(shop_df[col].dropna().unique()[:3].tolist()),
        "business_meaning": get_business_meaning_vietnamese(col),
        "extraction_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    dict_data.append(col_info)

data_dictionary = pd.DataFrame(dict_data)

# Hiển thị Data Dictionary
print(f"Generated Data Dictionary for {len(data_dictionary)} columns")
print("\n=== DATA DICTIONARY ===")
print(data_dictionary)

# Lưu vào file Excel
excel_path = "Technical_Document/Gold_dictionary.xlsx"

try:
    from openpyxl import load_workbook
    
    # Kiểm tra file Excel có tồn tại không
    try:
        # Load workbook hiện tại
        wb = load_workbook(excel_path)
        
        # Lấy sheet đầu tiên
        ws = wb.active
        
        # Kiểm tra xem có dữ liệu cũ không
        if ws.max_row > 1:
            print(f"Found existing data in {excel_path}, appending new data...")
        else:
            print(f"File {excel_path} exists but is empty, adding header and data...")
            
    except FileNotFoundError:
        print(f"File {excel_path} not found, creating new file...")
        wb = None
    except Exception as e:
        print(f"Error loading {excel_path}: {str(e)}, creating new file...")
        wb = None
    
    if wb is None:
        # Tạo file mới với header
        data_dictionary.to_excel(excel_path, index=False, sheet_name='Gold_Dictionary')
        print(f"✅ Created new file: {excel_path}")
    else:
        # Append vào file hiện tại
        from openpyxl.utils.dataframe import dataframe_to_rows
        
        # Tìm dòng cuối cùng có dữ liệu
        last_row = ws.max_row
        
        # Thêm dữ liệu mới từ dòng tiếp theo
        for r in dataframe_to_rows(data_dictionary, index=False, header=False):
            last_row += 1
            for c_idx, value in enumerate(r, 1):
                ws.cell(row=last_row, column=c_idx, value=value)
        
        # Lưu file
        wb.save(excel_path)
        print(f"✅ Appended {len(data_dictionary)} rows to: {excel_path}")
    
except Exception as e:
    print(f"❌ Error appending to Data Dictionary: {str(e)}")
    # Fallback: tạo file mới
    try:
        data_dictionary.to_excel(excel_path, index=False)
        print(f"✅ Created new file as fallback: {excel_path}")
    except Exception as e2:
        print(f"❌ Error creating fallback file: {str(e2)}")

# Summary Report
print(f"\n=== GOLD DICTIONARY SUMMARY ===")
print(f"Table: dim_shop")
print(f"Total columns: {len(data_dictionary)}")
print(f"Data Dictionary: {excel_path}")
print(f"Created at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n=== SAMPLE DICTIONARY ENTRIES ===")
sample_dict = data_dictionary[['column_name', 'dtype', 'business_meaning']].head(5)
print(sample_dict)


=== CREATING GOLD DATA DICTIONARY ===
Generated Data Dictionary for 4 columns

=== DATA DICTIONARY ===
  table_name column_name   dtype      sql_type  null_count  null_pct  \
0   dim_shop     shop_id   int64  VARCHAR(100)           0       0.0   
1   dim_shop   shop_name  object  VARCHAR(255)           0       0.0   
2   dim_shop    currency  object   Not defined           0       0.0   
3   dim_shop  avatar_url  object   Not defined           0       0.0   

   unique_count                                      sample_values  \
0             1                                        [230361475]   
1             1                                   ['Winner Group']   
2             1                                            ['VND']   
3             1  ['https://content.pancake.vn/2-24/2024/2/4/e0b...   

                              business_meaning      extraction_date  
0  Mã định danh cửa hàng duy nhất (khóa chính)  2025-10-18 00:16:42  
1                                 Tên cửa hàn